In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# hyperparameters
block_size = 64
batch_size = 16
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 100
n_embd = 64
n_layers = 4
n_head = 4
dropout = 0
learning_rate = 1e-3

In [29]:
text = open('../data/shakespeare.txt', 'r', encoding='utf-8').read()

In [30]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

# create character to token mapping
stoi = { ch: i for i, ch in enumerate(chars) }
itos = { i: ch for i, ch in enumerate(chars) }
encode = lambda s : [stoi[ch] for ch in s]
decode = lambda l : ''.join([itos[i] for i in l])

In [31]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [32]:
def get_batch(split):
    
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    X = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+1+block_size] for i in ix])
    X, y = X.to(device), y.to(device)
    return X, y

In [33]:
@torch.no_grad()
def estimate_loss(model):
    model.eval()
    out = {}
    for split in ['train', 'test']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, y = get_batch(split)
            logits, loss = model(X, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [34]:
class Head(nn.Module):
    
    def __init__(self, head_size):
        super().__init__()
        
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        B,T,C = x.shape
        
        k = self.key(x)
        q = self.query(x)
        # compute attention scores (affinities)
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        # values
        v = self.value(x)
        out = wei @ v
        return out

In [35]:
class MultiHeadAttention(nn.Module):
    
    def __init__(self, head_size, n_head):
        super().__init__()
        
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_head)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

In [36]:
class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout)
        )
        
    def forward(self, x):
        x = self.net(x)
        return x

In [37]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(head_size, n_head)
        self.ffwd = FeedForward()
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [38]:
class Transformer(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        
        B, T = idx.shape
        
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T).to(device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss

In [56]:
model = Transformer()
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/ 1e6, 'M parameters')
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

myIter = iter(model.parameters())
next(myIter)

0.211777 M parameters


Parameter containing:
tensor([[ 0.4768, -0.3815,  1.7805,  ...,  0.0212,  1.5440, -0.7566],
        [-0.1677, -0.4770, -0.4270,  ...,  0.2717,  0.7813, -0.4813],
        [-0.5074,  1.1178,  0.3616,  ...,  1.1994, -0.3860, -1.5267],
        ...,
        [-1.7375, -0.3586, -0.1701,  ...,  0.5683,  1.3694,  1.4239],
        [ 0.8873,  0.1602, -0.3562,  ..., -0.2543, -0.8033, -1.9426],
        [-1.0852, -1.1170,  0.2291,  ...,  1.3080,  0.3474,  0.5902]],
       device='cuda:0', requires_grad=True)

In [126]:
next(myIter).numel()

1024

In [51]:
epochs = 1000
for epoch in range(epochs):
    
    xb, yb = get_batch('train')
    
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step
    
    if epoch % (epochs / 10) == 0:
        print(loss.item())

4.450611591339111
4.410538196563721
4.416233062744141
4.438966751098633
4.462579727172852
4.464056968688965
4.464531421661377
4.456020832061768
4.43650484085083
4.456075668334961
